In [ ]:
import os
import shutil

# Thư mục gốc chứa tệp
#Cấp folder 1
source_folder = '../../classedExeImg'

# Thư mục con cần chia 
#Cấp folder 1
subfolder=['client1','client2','client3','client4','client5']

#biết rằng cấu trúc thư mục con giống với cấu trúc thư mục gốc ở cấp 2 và 3
#Cấp folder 2
type=['train','test','valid']
#Cấp folder 3
folder=['Locker', 'Mediyes', 'Winwebsec', 'Zbot', 'Zeroaccess','benign']
folder2=['Locker', 'Mediyes', 'Winwebsec', 'Zbot', 'Zeroaccess']

## Gộp 2 folder 

In [ ]:
# combine_folder=['train','test']
# #tạo thư mục temp để chứa tập dữ liệu
# temp_folder = '../../classedExeImg/Temp'
# if not os.path.exists(temp_folder):
#     os.makedirs(temp_folder)
# #gộp 2 tập dữ liệu train và test vào 1 thư mục temp
# for dataset in combine_folder:
#     for malware in folder:
#         source_folder_path = source_folder+'/'+dataset+'/'+malware
#         destination_folders = temp_folder+'/'+malware
#         if not os.path.exists(destination_folders):
#             os.makedirs(destination_folders)
#         file_list = os.listdir(source_folder_path)
#         for file in file_list:
#             source_path = os.path.join(source_folder_path, file)
#             dest_path = os.path.join(destination_folders, file)
#             shutil.copy(source_path, dest_path)

In [ ]:
benign_folder = '../../classedExeImg/Temp/benign'
folder_move='../../classedExeImg/valid/benign'

#Lấy ra 100 file benign để chuyển vào thư mục valid
benign_file_list = os.listdir(benign_folder)
for i in range(100):
    source_path = os.path.join(benign_folder, benign_file_list[i])
    dest_path = os.path.join(folder_move, benign_file_list[i])
    shutil.move(source_path, dest_path)
    print('Moved file:', benign_file_list[i])

### Lấy folder được gộp và tiến hành chia ra làm 5 folder nhỏ với train/test tỉ lệ 9/1

In [ ]:
temp_folder = '../../classedExeImg/Temp'
for malware in folder:
    source_folder_path = temp_folder+'/'+malware
    file_list = os.listdir(source_folder_path)
    #Chia tỉ lệ train/test là 9/1 từ file_list bằng train test split
    train_length = int(len(file_list) * 0.9)
    train_list = file_list[:train_length]
    test_list = file_list[train_length:]
    #Lấy số lượng mẫu dữ liệu train và test cho mỗi client
    train_per_folder = len(train_list) // len(subfolder)
    test_per_folder = len(test_list) // len(subfolder)
    for client in subfolder:
        #Tạo thư mục train và test cho mỗi client
        des_folder="../../ClientData/"
        train_folder = des_folder+client+'/train/'+malware
        test_folder = des_folder+client+'/test/'+malware
        if not os.path.exists(train_folder):
            os.makedirs(train_folder)
        if not os.path.exists(test_folder):
            os.makedirs(test_folder)
        #Copy tập dữ liệu train và test vào thư mục của client
        for i in range(train_per_folder):
            file=train_list.pop(0)
            source_path = os.path.join(source_folder_path, file)
            dest_path = os.path.join(train_folder, file)
            shutil.copy(source_path, dest_path)
        for i in range(test_per_folder):
            file=test_list.pop(0)
            source_path = os.path.join(source_folder_path, file)
            dest_path = os.path.join(test_folder, file)
            shutil.copy(source_path, dest_path)

## Chia dữ liệu non iid, 1 client có n class

In [ ]:
temp_folder = '../../classedExeImg/Temp'
target_folder = '../../NonIIDData'
#1/ Cho 1 mảng chứa số lượng phần tử tương ứng với n class có trong 1 client
#Ví dụ len(arr)=2 thì sẽ chia thành 2 class cho 1 client,

#2/ Dựa vào len(arr) này ta sẽ lấy số lượng ảnh trong 1 class của temp_folder
#Ví dụ len(arr)=n thì 1 client sẽ có được tối đa n class và 
#số lượng ảnh trong 1 class của client sẽ là len(số lượng ảnh trong 1 class của temp_folder)/n

#3/ Mỗi phần tử trong mảng arr là mảng chứa class tương ứng cho folder 1 client
#VD client 1 thì nếu len(arr[0])=2 thì client 1 sẽ có 2 class tương ứng
#là arr[0][0] và arr[1][0], client 2 sẽ có 2 class tương ứng là arr[0][1] và arr[1][1]...
#và giá trị bên trong mảng là chỉ số của class trong folder
#0 là Locker, 1 là Mediyes, 2 là Winwebsec, 3 là Zbot, 4 là Zeroaccess
arr=[
    [0,1,2,3,4],
    [4,0,1,2,3],
]
#Đầu tiên tạo ra các thư mục subfolder cho client trong target_folder
#và không có train, test, valid mà chỉ có các class 
#và số lượng class trong mỗi client sẽ là len(arr)
for client in subfolder:
    for i in range(len(arr)):
        client_folder = target_folder+'/'+client+'/'+folder[arr[i][subfolder.index(client)]]
        if not os.path.exists(client_folder):
            os.makedirs(client_folder)
            
#Sau đó chia ảnh cho client
for malware in folder:
    source_folder_path = temp_folder+'/'+malware
    file_list = os.listdir(source_folder_path)
    #Lấy số lượng ảnh cần chia cho client
    client_length = len(file_list)  // len(arr)
    for client in subfolder:
        #Lấy ra tên các folder con trong thư mục của client
        client_folder = target_folder+'/'+client
        list_folder = os.listdir(client_folder)
        #Kiểm tra xem client có chứa folder class đó không
        if malware in list_folder:
            print(list_folder)
            #Chia ảnh cho client
            for j in range(client_length):
                file=file_list.pop(0)
                source_path = os.path.join(source_folder_path, file)
                dest_path = os.path.join(client_folder+'/'+malware, file)
                shutil.copy(source_path, dest_path)
            continue
                

In [28]:
a=900//3
b=round(a*0.9)
c=a-b
print(a,b,c)

300 270 30


## Chia data cho detection

In [24]:
def copy_files(temp_folder, client_folder,type, label, file):
    source_path = os.path.join(temp_folder, file)
    dir_path = os.path.join(client_folder+type+label)
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
    dest_path = os.path.join(client_folder+type+label, file)
    shutil.copy(source_path, dest_path)

In [27]:
#Lấy ra tất cả các ảnh từ thư mục Temp chứa 5 class malware
#Bỏ vào thư mục Malware (chỉ lấy ảnh bỏ vào thư mục Malware, không lấy folder bỏ vô)
temp_folder = '../../classedExeImg/Malware'
benign='../../classedExeImg/Temp/benign'

#Lấy hết file ảnh từ thư mục temp và sort theo size
#sau đó chia làm 3 phần, mỗi phần tương ứng với 1 client
#trong đó tiếp tục lấy 3 phần đó, trong mỗi phần sẽ được chia theo tỉ lệ 9/1
#với 9 phần là train và 1 phần là test (chia ra thành thư mục train và test)
#Sau đó làm tương tự với thư mục benign
#Cuối cùng sẽ được thư mục NonIIDDataDetect có 5 client 
# 2 folder client4,client5 chỉ chứa 1 class benign
# 2 folder client1,client2 chỉ chứa các ảnh từ 5 class malware
# 1 folder client3 chứa cả 5 class malware và 1 class benign
#Với tỉ lệ 9/1

target_folder = '../../NonIIDDataDetect'
#tạo 5 thư mục client
for client in subfolder:
    client_folder = target_folder+'/'+client
    if not os.path.exists(client_folder):
        os.makedirs(client_folder)

list_file = os.listdir(temp_folder)
#Sắp xếp theo size
list_file = sorted(list_file, key=lambda x: os.path.getsize(os.path.join(temp_folder, x)))
#Sort theo size và chia làm 2 phần
#phần đầu giữ lại, phần sau sort theo tên
list_file_1 = list_file[int(len(list_file)/2):]
list_file_2 = sorted(list_file[:int(len(list_file)/2)])
list_file = list_file_1 + list_file_2
print(len(list_file))
#Chia thành 3 phần
part_length = len(list_file) // 3
part1 = list_file[:part_length]
part2 = list_file[part_length:2*part_length]
part3 = list_file[2*part_length:]
#Chia thành 2 phần train và test
train_length = int(part_length * 0.9)
train1 = part1[:train_length]
test1 = part1[train_length:]
train2 = part2[:train_length]
test2 = part2[train_length:]
train3 = part3[:train_length]
test3 = part3[train_length:]
#Chia thành 2 phần benign và malware
benign_list = os.listdir(benign)
benign_length = len(benign_list) // 3
benign1 = benign_list[:benign_length]
benign2 = benign_list[benign_length:2*benign_length]
benign3 = benign_list[2*benign_length:]
#Chia thành 2 phần train và test
train_benign_length = int(benign_length * 0.9)
train_benign1 = benign1[:train_benign_length]
test_benign1 = benign1[train_benign_length:]
train_benign2 = benign2[:train_benign_length]
test_benign2 = benign2[train_benign_length:]
train_benign3 = benign3[:train_benign_length]
test_benign3 = benign3[train_benign_length:]
label=['/malware','/benign']
#bỏ 3 phần malware vào 3 client client1,client2,client3
for client in subfolder:
    client_folder = target_folder+'/'+client
    if not os.path.exists(client_folder+'/train'):
        os.makedirs(client_folder+'/train')
    if not os.path.exists(client_folder+'/test'):
        os.makedirs(client_folder+'/test')
    if client=='client1':
        for file in train1:
            copy_files(temp_folder, client_folder, '/train', label[0], file)
        for file in test1:
            copy_files(temp_folder, client_folder, '/test', label[0], file)
    if client=='client2':
        for file in train2:
            copy_files(temp_folder, client_folder, '/train', label[0], file)
        for file in test2:
            copy_files(temp_folder, client_folder, '/test', label[0], file)
    if client=='client3':
        for file in train3:
            copy_files(temp_folder, client_folder, '/train', label[0], file)
        for file in test3:
            copy_files(temp_folder, client_folder, '/test', label[0], file)
        for file in train_benign1:
            copy_files(benign, client_folder, '/train', label[1], file)
        for file in test_benign1:
            copy_files(benign, client_folder, '/test', label[1], file)
    if client=='client4':
        for file in train_benign2:
            copy_files(benign, client_folder, '/train', label[1], file)
        for file in test_benign2:
            copy_files(benign, client_folder, '/test', label[1], file)
    if client=='client5':
        for file in train_benign3:
            copy_files(benign, client_folder, '/train', label[1], file)
        for file in test_benign3:
            copy_files(benign, client_folder, '/test', label[1], file)

8074


### Lấy 3 folder train, test, valid tổng chia ra làm 5 phần bằng nhau

In [ ]:
# for dataset in type:
#     for malware in folder:
#         source_folder_path = source_folder+'/'+dataset+'/'+malware
#         #Lấy số lượng cần chia cho mỗi folder
#         files_per_folder = len(os.listdir(source_folder_path)) // 5
#         file_list = os.listdir(source_folder_path)
#         for client in subfolder:
#             destination_folders ='./FederatedData/'+client+'/'+dataset+'/'+malware
#             if(client=='client5'):
#                 files_per_folder = len(os.listdir(source_folder_path)) - (4 * files_per_folder)
#             for _ in range(files_per_folder):
#                 file = file_list.pop(0)
#                 source_path = os.path.join(source_folder_path, file)
#                 dest_path = os.path.join(destination_folders, file)
#                 shutil.copy(source_path, dest_path)